# Hybrid Search Test

In [1]:
# OpenAI API 설정 및 라이브러리 import
from openai import OpenAI
import json
import os
from typing import List, Dict
import dotenv
import sys
import os

# 프로젝트 루트 디렉토리를 Python 경로에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)


dotenv.load_dotenv()
# API 키 설정 (환경변수에서 가져오기)
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")  # 환경변수에 API 키를 설정해주세요
)

def chat(
    messages: List[Dict[str, str]], 
    model: str = "gpt-4o-mini",
    **kwargs
) -> str:
    """
    OpenAI GPT-4o-mini API를 사용하여 채팅 완성을 수행합니다.
    
    Args:
        messages: 대화 메시지 리스트 [{"role": "user", "content": "메시지"}]
        model: 사용할 모델명 (기본값: gpt-4o-mini)
        **kwargs: OpenAI API 매개변수들
            - temperature: 창의성 조절 (0.0-2.0, 기본값: 0.7)
            - max_tokens: 최대 토큰 수 (기본값: 1000)
            - top_p: 확률 임계값 (기본값: 0.95)
            - frequency_penalty: 빈도 페널티 (기본값: 0.0)
            - presence_penalty: 존재 페널티 (기본값: 0.0)
            - stream: 스트리밍 여부 (기본값: False)
            - 기타 OpenAI API가 지원하는 모든 매개변수
        
    Returns:
        GPT 응답 텍스트
    """
    # 기본값 설정
    default_params = {
        "temperature": 0.7,
        "max_tokens": 1000,
        "top_p": 0.95,
        "frequency_penalty": 0.0,
        "presence_penalty": 0.0
    }
    
    # 기본값과 사용자 입력 병합
    params = {**default_params, **kwargs}
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            **params
        )
        
        return response.choices[0].message.content
        
    except Exception as e:
        print(f"API 호출 중 오류 발생: {e}")
        return None


In [2]:
from prompts.prompts import load_prompt

translate_prompt = load_prompt("translate")
print(translate_prompt)

당신은 "외국인 노동자가 한국에서 겪는 고충을 해결해주는 상담 챗봇" 도메인에 특화된 번역자이자 키워드 추출자입니다. 사용자 쿼리(다국어)를 다음과 같은 순서로 처리하고, 오직 JSON 형식으로만 응답해야 합니다:
## 역할
1. 다국어로 입력된 문장을 정확하고 자연스럽게 한국어로 번역
2. 번역된 문장에서 DB 검색을 위한 한국어 키워드를 압축 추출

## 지시사항 (반드시 준수)
1. 원본 쿼리를 “text” 필드에 기록합니다.
2. 이를 한국어로 자연스럽고 정확하게 번역하여 “translated” 필드에 기록합니다.
3. 번역된 문장에서 **DB 검색을 위해 유용한 핵심 키워드**를 한국어로 추출하여 “keyword” 필드의 리스트에 넣습니다.
출력 형식:JSON만 출력. 설명/라벨/마크업/추가 텍스트 금지.
```json
{
  "text": "입력 문장",
  "translated": "번역된 문장",
  "keyword": ["키워드1", "키워드2", "키워드3"]
}
```

### 키워드 추출 지시사항
목적: 실제 DB 검색에 쓰일 핵심 용어만 뽑기.
형태: 한국어, 1~3단어의 단위로 2~6개 내.
- 일반적이고 넓은 의미의 단어(예: “한국”, “노동”)는 제외
- 키워드는 DB에서 실제 검색에 사용될 수 있도록 구체적이어야 함
우선순위:
비자/제도명: 예) H-2, 특례고용허가제, E-9
행정행위/절차: 예) 사업장변경, 체류자격변경, 신고의무, 필요서류
분쟁·권리 키워드: 예) 임금체불, 산재, 부당해고, 근로계약
취업·업종/지역이 구체적일 때만: 예) 제조업, 건설업, 농축산업 등
정규화: 동의어·구어는 표준 용어로 통합(예: 일자리 옮기기 → 사업장변경)
원문이 영어/스페인어 등이어도 키워드는 한국어로


## 예시
Input: `"Xin chào, tôi là kiều bào đang sinh sống tại Việt Nam. Lần này, tôi muốn xin việc tại Hàn Quốc thông qua Chế độ cấ

In [ ]:
# messages = [
#     {"role": "system", "content": translate_prompt},
#     {"role": "user", "content": "안녕하세요! 파이썬에 대해 간단히 설명해주세요."}
# ]

# response = chat(messages)

### 테스트

In [3]:
import pandas as pd

query_df = pd.read_csv("../data/helloworld_test_query.csv")

In [4]:
# CSV 파일 불러오기 및 데이터프레임 생성
import pandas as pd

# CSV 파일 불러오기
query_df = pd.read_csv('../data/helloworld_test_query.csv')
print(f"불러온 데이터 개수: {len(query_df)}")
print("\n데이터프레임 컬럼:")
print(query_df.columns.tolist())
print("\n첫 3개 행:")
print(query_df.head(3))


불러온 데이터 개수: 20

데이터프레임 컬럼:
['query', 'translated_query', '언어', 'ground_truth_id', 'category', 'source', '작성자', '비고', '소스']

첫 3개 행:
                                               query  \
0  제 여자친구가 단속으로 출입국 보호소에 있습니다. 월급이 아직 들어오지 않았는데, ...   
1  안녕하세요, 건설 현장에서 일하고 있는 사람인데, 사장님이 월급을 안줘서 계좌가 압...   
2  안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 근무를 하고 이제 제 나라...   

                                    translated_query            언어  \
0  แฟนของผมถูกจับกุมและอยู่ที่ศูนย์กักตัวตรวจคนเข...           태국어   
1     你好，我是在建筑工地工作的，但老板没有发工资，我的账户可能会被查封。遇到这种情况该怎么办呢？           중국어   
2  Magandang araw, ako ay isang manggagawang mula...  필리핀어 (타갈로그어)   

                                     ground_truth_id category  \
0  ['689b3a86ffd306c1cd3c09a4', '689b3a86ffd306c1...     임금체불   
1  ['689b3a86ffd306c1cd3c06e8', '689b3a86ffd306c1...     임금체불   
2                       ['689b3a86ffd306c1cd3c09a4']     임금체불   

            source  작성자                                       비고  \
0  경기도외국인지원센터_상담사례  황예원          

In [5]:
# 번역 프롬프트 가져오기
from prompts.prompts import load_prompt

translate_prompt = load_prompt("translate")
print("번역 프롬프트 로드 완료")

# 번역 함수 정의
def translate_query(query_text):
    """번역 쿼리를 GPT-4o-mini로 번역하여 JSON 응답을 받는다"""
    messages = [
        {"role": "system", "content": translate_prompt},
        {"role": "user", "content": query_text}
    ]
    
    try:
        response = chat(messages, temperature=0.1)
        result = json.loads(response)
        return result
    except Exception as e:
        print(f"번역 오류: {e}")
        return None


번역 프롬프트 로드 완료


In [6]:
# 모든 쿼리 처리
import json
from tqdm import tqdm
import time

# 결과를 저장할 리스트
all_results = []
translated_results = []
keyword_results = []

print(f"총 {len(query_df)}개의 쿼리를 처리합니다...")

# 각 쿼리 처리
for idx, row in tqdm(query_df.iterrows(), total=len(query_df), desc="번역 진행"):
    query_text = row['translated_query']
    
    # 번역 수행
    result = translate_query(query_text)
    
    if result is not None:
        # 전체 결과 저장
        all_results.append(result)
        
        # 개별 필드 저장
        translated_results.append(result.get('translated', ''))
        keyword_results.append(result.get('keyword', []))
        
        print(f"[{idx+1}/{len(query_df)}] 완료: {result.get('translated', '')[:50]}...")
    else:
        # 오류 발생 시 빈 값 추가
        all_results.append(None)
        translated_results.append('')
        keyword_results.append([])
        print(f"[{idx+1}/{len(query_df)}] 오류 발생")
    
    # API 호출 제한을 위한 잠시 대기
    time.sleep(0.5)

print(f"\n번역 완료! 총 {len([r for r in all_results if r is not None])}개 성공")
print(f"오류 발생: {len([r for r in all_results if r is None])}개")


총 20개의 쿼리를 처리합니다...


번역 진행:   0%|          | 0/20 [00:00<?, ?it/s]

[1/20] 완료: 제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는데, 만...


번역 진행:   5%|▌         | 1/20 [00:04<01:20,  4.22s/it]

[2/20] 완료: 안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 월급을 주지 않아서 제 계좌가 압류...


...

[19/20] 완료: 해외 출국 만기 보험금을 신청하려면 어떤 자격 조건을 충족해야 하나요? 언제부터 신청할 수...


번역 진행:  95%|█████████▌| 19/20 [01:13<00:03,  3.90s/it]

[20/20] 완료: 고국으로 돌아갈 때, 언제부터 귀국비용 보험을 신청할 수 있으며, 어떤 서류를 준비해야 하...


번역 진행: 100%|██████████| 20/20 [01:15<00:00,  3.79s/it]


번역 완료! 총 20개 성공
오류 발생: 0개


In [ ]:
# 결과 저장
import datetime

# 1. JSONL 파일로 전체 결과 저장
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
jsonl_filename = f"../data/translation_results_{timestamp}.jsonl"

print(f"JSONL 파일 저장: {jsonl_filename}")

with open(jsonl_filename, 'w', encoding='utf-8') as f:
    for result in all_results:
        if result is not None:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
        else:
            f.write(json.dumps({"error": "translation_failed"}, ensure_ascii=False) + '\n')

print(f"JSONL 파일 저장 완료: {len(all_results)}개 레코드")

# 2. 데이터프레임에 새 컬럼 추가
query_df['translated_4o_mini'] = translated_results
query_df['keyword_4o_mini'] = keyword_results

print("\n데이터프레임 새 컬럼 추가 완료:")
print(f"- translated_4o_mini: {len(translated_results)}개")
print(f"- keyword_4o_mini: {len(keyword_results)}개")

# 3. 업데이트된 CSV 저장
updated_csv_filename = f"../data/helloworld_test_query_with_translation_{timestamp}.csv"
query_df.to_csv(updated_csv_filename, index=False, encoding='utf-8')

print(f"\n업데이트된 CSV 파일 저장: {updated_csv_filename}")


JSONL 파일 저장: ../data/translation_results_20250910_010300.jsonl
JSONL 파일 저장 완료: 20개 레코드

데이터프레임 새 컬럼 추가 완료:
- translated_4o_mini: 20개
- keyword_4o_mini: 20개

업데이트된 CSV 파일 저장: ../data/helloworld_test_query_with_translation_20250910_010300.csv

결과 미리보기:
                                    translated_query  \
0  แฟนของผมถูกจับกุมและอยู่ที่ศูนย์กักตัวตรวจคนเข...   
1     你好，我是在建筑工地工作的，但老板没有发工资，我的账户可能会被查封。遇到这种情况该怎么办呢？   
2  Magandang araw, ako ay isang manggagawang mula...   

                                  translated_4o_mini  \
0  제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는...   
1  안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 월급을 주지 않아서 제 계좌...   
2  안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 일했는데 이제 고국으로 돌...   

               keyword_4o_mini  
0  [체포, 출입국 관리소, 밀린 월급, 태국 귀국]  
1           [건설업, 임금체불, 계좌 압류]  
2                [임금체불, 회수 방법]  


In [9]:
query_df[['query', 'translated_4o_mini', 'keyword_4o_mini']]

,query,translated_4o_mini,keyword_4o_mini
0,"제 여자친구가 단속으로 출입국 보호소에 있습니다. 월급이 아직 들어오지 않았는데, ...",제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는...,"[체포, 출입국 관리소, 밀린 월급, 태국 귀국]"
1,"안녕하세요, 건설 현장에서 일하고 있는 사람인데, 사장님이 월급을 안줘서 계좌가 압...","안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 월급을 주지 않아서 제 계좌...","[건설업, 임금체불, 계좌 압류]"
2,"안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 근무를 하고 이제 제 나라...","안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 일했는데 이제 고국으로 돌...","[임금체불, 회수 방법]"
3,사업장 변경 신청 이후 제가 불법체류자가 될 수 있다는 우편이 날아왔어요. 8월 2...,"근무지 변경 신청 후, 불법 체류자가 될 수 있다는 내용의 편지를 받았습니다. 8월...","[근무지 변경, 불법 체류, 추방]"
4,"제가 중간에 퇴직을 하게 되었는데, 소득세가 체납되어 비자 연장이 안된대요. 그런데...","저는 중간에 퇴사했지만, 소득세 체납 때문에 비자를 연장할 수 없습니다. 그런데 관...","[퇴사, 소득세 체납, 비자 연장, 통지 내용]"
5,"회사에서 갑자기 나오지 말라고 해서, 체류 자격이 박탈될 것 같습니다. 저는 어떻게...","회사가 갑자기 더 이상 출근하지 말라고 해서, 체류 자격이 박탈될 수 있습니다. 어...","[체류 자격 박탈, 대처 방법]"
6,"건설업에 근무 중인 외국인입니다. 어떤 경우에 산업재해가 인정되는지, 또 경우에 따...",저는 건설업에서 일하고 있는 외국인 노동자입니다. 어떤 경우가 산업재해로 간주되는지...,"[외국인 노동자, 건설업, 산업재해, 보상]"
7,"안녕하세요, 저는 베트남에 거주중인 재외동포입니다. 이번에 특례고용허가제를 통해 한...","안녕하세요, 저는 베트남에 거주 중인 재외동포입니다. 이번에 특례고용허가제를 통해 ...","[재외동포, 특례고용허가제, 절차]"
8,"H-2 비자를 가지고 있는데, 지금 일하고 있는 사업주를 떠나 다른 사업장으로 옮길...","저는 H-2 비자를 가지고 있는데, 현재 고용주를 떠나 다른 직장으로 옮길 수 있는...","[H-2 비자, 사업장 변경, 절차, 지원 서류]"
9,고용주가 지속적으로 임금체불을 하여 사업장 변경을 원합니다.,고용주가 지속적으로 임금을 체불하여 저는 근무지를 옮기고 싶습니다.,"[임금체불, 근무지 변경]"


In [10]:
for i, row in query_df.iterrows():
    print(row['query'])
    print(row['translated_4o_mini'])
    print(row['keyword_4o_mini'])
    print('-'*100)

제 여자친구가 단속으로 출입국 보호소에 있습니다. 월급이 아직 들어오지 않았는데, 태국으로 가버리면 밀린 월급은 어떻게 받죠..
제 여자친구가 체포되어 출입국 관리소에 구금되어 있습니다. 지금 월급을 받지 못했는데, 만약 태국으로 돌아가야 한다면 밀린 월급을 어떻게 받을 수 있을까요?
['체포', '출입국 관리소', '밀린 월급', '태국 귀국']
----------------------------------------------------------------------------------------------------
안녕하세요, 건설 현장에서 일하고 있는 사람인데, 사장님이 월급을 안줘서 계좌가 압류될 것 같습니다. 이런 경우 어떻게 해야 하나요?
안녕하세요, 저는 건설 현장에서 일하고 있는데, 사장이 월급을 주지 않아서 제 계좌가 압류될 수 있습니다. 이런 상황에서는 어떻게 해야 하나요?
['건설업', '임금체불', '계좌 압류']
----------------------------------------------------------------------------------------------------
안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 근무를 하고 이제 제 나라로 돌아가려 해요. 그런데 회사가 망해서 월급을 못받았습니다. 제 월급을 돌려받을 방법이 없나요?
안녕하세요, 저는 필리핀에서 온 노동자입니다. 5년 동안 일했는데 이제 고국으로 돌아가려고 합니다. 그런데 회사가 망해서 제 임금을 받지 못했습니다. 제 임금을 회수할 방법이 정말 없는 건가요?
['임금체불', '회수 방법']
----------------------------------------------------------------------------------------------------
사업장 변경 신청 이후 제가 불법체류자가 될 수 있다는 우편이 날아왔어요. 8월 22일까지만 체류가 허가되어있고, 9월부터는 쫓겨날 수도 있대요. 어떡하죠?
근무지 